In [1]:
import os

import numpy as np

import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt

In [2]:
weights = DataFrame()
file = 'scores.xlsx'

raw_product = pd.read_excel(file, sheet_name='Product', skiprows=2, usecols=3)
raw_company = pd.read_excel(file, sheet_name='Company', skiprows=2, usecols=3)

raw = raw_product.append(raw_company)
raw = raw.reset_index(drop=True)
raw.columns = ['requirement', 'score', 'notes', 'weight']
del raw['score']
del raw['notes']


# starting at the last category
# extract the rows for the category to the end of the df
# remove the rows
header_indexes = raw.index[raw['requirement'] == 'Requirement'].tolist()
header_indexes = list(reversed(header_indexes))

for hi in header_indexes:
    category = raw.iloc[hi-1][0]
    sub_weights = DataFrame(raw.iloc[hi+1:])
    sub_weights['category'] = category
    sub_weights = sub_weights[pd.notnull(sub_weights['requirement'])]

    weights = weights.append(sub_weights)
    raw = raw.iloc[0:hi-1]

    
weights = weights.reset_index(drop=True)
weights

,requirement,weight,category
0,D&B Rating,5,Corporate Viability
1,Years in business,1,Corporate Viability
2,Hold time less than 5 minutes,5,Technical Support
3,Resolution on first call,5,Technical Support
4,Polite,2,Technical Support
5,Pleasant apppearance,1,Packaging
6,Easy to open,1,Packaging
7,Recyclable,2,Packaging
8,Must be able to hold 20 oz,5,Technical Specs
9,Flexible,2,Technical Specs


In [3]:
scores = DataFrame()
files = [f for f in os.listdir('.') if f.endswith('-scores.xlsx')]

for file in files:
    raw = pd.read_excel(file, sheet_name='Product', usecols=1, header=None)
    scorer, product = raw.iloc[0][1], raw.iloc[1][1]

    raw_product = pd.read_excel(file, sheet_name='Product', skiprows=2, usecols=2)
    raw_company = pd.read_excel(file, sheet_name='Company', skiprows=2, usecols=2)

    raw = raw_product.append(raw_company)
    raw = raw.reset_index(drop=True)
    raw.columns = ['requirement', 'score', 'notes']

    # starting at the last category
    # extract the rows for the category to the end of the df
    # remove the rows
    header_indexes = raw.index[raw['requirement'] == 'Requirement'].tolist()
    header_indexes = list(reversed(header_indexes))

    for hi in header_indexes:
        category = raw.iloc[hi-1][0]
        sub_scores = DataFrame(raw.iloc[hi+1:])
        sub_scores['category'] = category
        sub_scores['scorer'] = scorer
        sub_scores['product'] = product
        sub_scores = sub_scores[pd.notnull(sub_scores['requirement'])]

        scores = scores.append(sub_scores)
        raw = raw.iloc[0:hi-1]

    
scores = scores.reset_index(drop=True)
scores = pd.merge(scores, weights, on=['category', 'requirement'])
scores


,requirement,score,notes,category,scorer,product,weight
0,D&B Rating,4,NaN,Corporate Viability,Jane Doe,Acme Widget,5
1,D&B Rating,3,NaN,Corporate Viability,Jane Doe,Romco Widget,5
2,D&B Rating,3,NaN,Corporate Viability,John Doe,Acme Widget,5
3,D&B Rating,2,NaN,Corporate Viability,John Doe,Romco Widget,5
4,Years in business,5,NaN,Corporate Viability,Jane Doe,Acme Widget,1
5,Years in business,3,NaN,Corporate Viability,Jane Doe,Romco Widget,1
6,Years in business,5,NaN,Corporate Viability,John Doe,Acme Widget,1
7,Years in business,2,NaN,Corporate Viability,John Doe,Romco Widget,1
8,Hold time less than 5 minutes,3,NaN,Technical Support,Jane Doe,Acme Widget,5
9,Hold time less than 5 minutes,2,NaN,Technical Support,Jane Doe,Romco Widget,5
